## 1. 用gauss拟合得到峰位和sigma

按照下图图示，运行命令
![](com1.png)

执行`h0->Draw()`，按照下图图示，在左侧空白处点击鼠标右键，在弹出菜单中选择`SetLogy`将y轴由线性坐标改成指数坐标，
![](2a.png)

可得到
![](0.png)

上图为未刻度的gamma谱，其中横轴为道址，纵轴为计数。进行刻度时参照刻度方法中给出的$^{152}Eu$和$^{133}Ba$能谱，查找到对应峰位进行刻度。

为获得gamma峰峰位，需要进行拟合操作。

以330-335道附近的峰为例，演示下如何用root拟合峰位。
![](2.png)

鼠标悬浮于放在峰位上，右键选择fitpanel。![](3.png) 出现fitpanel![](4.png)

选择默认的gaus拟合即可。拖动下方滑块选择合适的拟合区间。随后点击fit即可完成拟合。![](5.png)

可在终端中查看拟合参数![](6.png)也可点击菜单栏 Options->Fit Parameters进行查看。![](7.png)

从而可以获得峰位及误差信息。这里需要留意，选取合适的拟合区间对于峰位的确认至关重要。上图中可适当再次缩小拟合区间，以获得更好的效果。![](8.png)

此时再从能谱中找出对应的能量，即可获得一组数据点。半宽可直接通过拟合获得的sigma获得。

## 2. 将数据填入TGraphErrors进行拟合。
第一章的作业中用到的TGraph，一般用来表达没有误差的二维数据点的分布。当数据有误差时需要用TGraphErrors，如以上拟合得到的峰位值有误差值。

仿照下面链接[TGraphErrors Class Reference](https://root.cern.ch/doc/master/classTGraphErrors.html)的做法，将数据(包含误差值)填入TGraphErrors,并画图。152Eu，133Ba的能量值[keV]的误差全部设置为0.
将鼠标指针悬浮于图上的一个点上，点击右键选择fitpanel，在fitfunction栏中选择p2(二次多项式)，进行拟合得到拟合系数。需要留意，在进行拟合时，需要勾选`Fit Options`中的`All weights=1`进行拟合得到合理的拟合参数初值(相当于忽略误差进行拟合), 然后去掉`All weights=1`的选择，再进行拟合得到最终的拟合值。
![](fitoption.png)

关于ROOT fit的更详细用法参考以下链接[FittingHistograms](https://root.cern.ch/root/htmldoc/guides/users-guide/FittingHistograms.html)

## 3. 应用获得的刻度系数，绘制$\gamma$能谱

在上一步获得能量刻度系数之后，应用刻度系数即可获得刻度后的能谱。然而直接对直方图进行变换会出现一些并道的问题，这里不做过多阐述。简化起见，提供如下示例代码，供大家应用刻度系数直接生成刻度后的能谱。

将以下代码存成 cal.C

```cpp
void cal()
{   
    double p0=0;//填入拟合得到的数值
    double p1=1;
    double p2=0;
    TH1D *g0=new TH1D("g0","g0",12500,0,2500);//g0为刻度后的gamma谱
    TRandom3 *r=new TRandom3(0);
    for(int i=0;i<12500;i++){
        //h0为未刻度的gamma谱。
        Long64_t eN=h0->GetBinContent(i);// GetBinContent()提取每个bin的计数
        Double_t e=h0->GetBinCenter(i);// GetBinCenter()提取每个bin的中心的横坐标x。
        for(Long64_t j=0;j<eN;j++){
            Double_t ea=e+r->Rndm()*0.2-0.1;
            ea=p2*ea*ea+p1*ea+p0;//p0,p1,p2为上面获得的二次项刻度系数。
            g0->Fill(ea);
        }
        if(i%1000==0) cout<<i<<endl;//显示运行进度
    }
    TFile *f=new TFile("gammacali.root","recreate");
    g0->Write();
    f->Close();
}
```
按照下图操作
![](com2.png)

如果运行正常，可看到`gammacali.root`文件生成。

按下图操作显示刻度好的能谱g0
![](com3.png)
